In [26]:
import requests
import json
import sys
import os
import time
import requests
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import load_dotenv
from fastapi.responses import JSONResponse

In [28]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

credentials = service_account.Credentials.from_service_account_file(
    "/mnt/c/Users/m_was/Downloads/tomastestproject-433206-48a55703dec2.json")
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
table_id = "tomastestproject-433206.testdb_1.table_6"  

def fetch_and_upload_stock_data(stock_symbol):
  
        # Construct the API URL
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&apikey={API_KEY}"
        response = requests.get(url=url)
        response.raise_for_status()
        data = response.json()
        if data:  # Check if the data is not empty
            time_series = data.get("Time Series (Daily)", {})  # Get the time series data
            rows_to_insert = []  # Initialize an empty list to store the data that will be inserted into BigQuery
            for date, daily_data in time_series.items():  # .items() returns a list of tuples with the key-value pairs of the dictionary
                row = {
                    "date": date,
                    "open": float(daily_data["1. open"]),
                    "high": float(daily_data["2. high"]),
                    "low": float(daily_data["3. low"]),
                    "close": float(daily_data["4. close"]),
                    "volume": int(daily_data["5. volume"]),
                    "stock_symbol": stock_symbol  # Use stock_symbol instead of undefined stock_name
                }
                rows_to_insert.append(row)

            # Insert rows into BigQuery
            errors = client.insert_rows_json(table_id, rows_to_insert)
            if errors == []:
                print("New rows have been added.")
            else:
                print(f"Encountered errors while inserting rows: {errors}")

FileNotFoundError: [Errno 2] No such file or directory

In [25]:
fetch_and_upload_stock_data("")

New rows have been added.


In [ ]:
def fetch_raw_stock_data(stock_symbol: str):
    try:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_symbol}&apikey={API_KEY}"
        response = requests.get(url=url)
        response.raise_for_status()
        data = response.json()

        if data:
            return JSONResponse(data)  # Return the raw JSON response
        else:
            return JSONResponse({"error": "Invalid or empty response from Alpha Vantage API"}, status_code=500)

    except requests.exceptions.RequestException as e:
        return JSONResponse({"error": f"An error occurred during the API request: {e}"}, status_code=500)
